# Creation Virtual Environment 

In [ ]:
# git clone https://github.com/carpentries-incubator/SDC-BIDS-fMRI.git

# module load NiaEnv/2019b python/3.6.8
# virtualenv --system-site-packages ~/.virtualenvs/sdc_bids_fmri
# source ~/.virtualenvs/sdc_bids_fmri/bin/activate

# cd SDC-BIDS-fMRI
# pip install -r requirements.txt
# pip install ipykernel
# python -m ipykernel install --user --name=sdc_bids_fmri

# #to create the link in for the dataset
# ln -s /scinet/course/ss2020/6_mripython/data/ds000030 ./data/

# https://localhost:8888/?token=fill the token number 

# Dataset Visualization - Tree

In [ ]:
!tree -L 1 '../data/derivative..'
#!tree

# Initial Setup


In [ ]:
import bids
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from nilearn import image as nimg
from nilearn import plotting as nplot
from bids.layout import BIDSLayout 
#from bids import BIDSLayout 
%matplotlib inline 

# Layout Setup

In [ ]:
fmriprep_dir = '../data/ds000030/derivatives/fmriprep/'
#also use pwd for fmriprep_dir

layout=BIDSLayout(fmriprep_dir, validate=False,
                  config=['bids','derivatives'])

T1w_files = layout.get(subject='10788',
                       datatype='anat', desc='preproc',
                       space='MNI152NLin2009cAsym',
                       extension='nii.gz',
                      return_type='file')

brainmask_files = layout.get(subject='10788',
                             datatype='anat', suffix='mask',
                             desc='brain',
                             space='MNI152NLin2009cAsym',
                             extension='nii.gz',
                            return_type='file')

func_files = layout.get(subject='10788',
                        datatype='func', desc='preproc',
                       space='MNI152NLin2009cAsym',
                       extension='nii.gz',
                       return_type='file')

func_mask_files = layout.get(subject='10788',
                             datatype='func', suffix='mask',
                             desc='brain',
                             space='MNI152NLin2009cAsym',
                             extension='nii.gz',
                            return_type='file')

#layout.get_subjects() #to get all subjects 

#layout.get_tasks() #to get task during mri taken, rest or doing something

# Image object Creation


In [ ]:
#structural image
T1w_mni = T1w_files[0]
T1w_mni_img = nimg.load_img(T1w_mni)

#functional image
func_mni = func_files[0]
func_mni_img = nimg.load_img(func_mni)

#brain mask
bm_mni = brainmask_files[0]
bm_mni_img = nimg.load_img(bm_mni)


# MRI IMAGE VISUALIZATION

## Anatomic Image Visualization 
### 1. simple image visualization


In [ ]:
#Structural plot 
nplot.plot_anat(T1w_mni_img)

# func image plot 
nplot.plot_epi(func_mni_img) #epi will show the clolor map
#nplot.plot_anat(func_mni_img)

#Brain mask plot 
nplot.plot_anat(bm_mni_img)

### 2. slider image visualization


In [ ]:
nplot.view_img(T1w_mni_img, bg_img = False,cmap = 'Grey_s', symmetry_cmap = False, threshold = "auto")

## Functional Image visualization 
### 1. simple image visualization


### 2. slider image visualization


### 3. Animated image visualization



In [ ]:
import matplotlib.animation
from IPython.display import HTML

%%capture
%matplotlib inline

#Resample the T1 to the size of the functional image!
resamp_t1 = nimg.resample_to_img(source_img=T1_mni_img, target_img=func_mni_img, interpolation='continuous')
fig, ax = plt.subplots()

def animate(image):
    nplot.plot_anat(image, figure=fig, cut_coords=(0,0,0))
    ax.set_facecolor('black')
    
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=[resamp_t1, T1_mni_img])
# ani = matplotlib.animation.FuncAnimation(fig, animate, frames=[resamp_t1, func])

# Display animation
HTML(ani.to_jshtml())

### 4. Time series plot 



In [ ]:
#preparing the single voxel

single_vox = func_mni_img.slicer[59:60,45:46,30:31,:].get_data()
single_vox.shape #just to show the shpae 

single_vox = single_vox.flatten() #to convert into 1D array
single_vox.shape #(152,) 1D array

# Mask image visualization



Problem solving

In [ ]:
#Structural Image 
#get the file of the structural image
T1_mni = T1w_files[0]
T1_mni_img = nimg.load_img(T1_mni)
print(f"T1w : {T1_mni_img.shape})

#get the file of the brain mask structureal image
bm_t1w = T1w_files[0]
bm_t1w_img = nimg.load_img(bm_t1w)
print(f"T1W brain mask : {bm_t1w_img.shape})
print(T1_mni_img.shape == bm_t1w_img.shape)


#Functional Image 
#get the file of the functional image
func_mni = T1w_files[0]
func_mni_img = nimg.load_img(func_mni)
##slicing functional image to 1st volume 
func_mni_img = func_mni_img.slicer[:,:,:,0]

#checking resting state 
print(f"func state : {func_mni.get_tasks()}")
print(f"func : {func_mni_img.shape})

#get the file of the functional brain mask image
bm_func = T1w_files[0]
bm_func_img = nimg.load_img(bm_func)
print(f"func brain mask : {bm_func_img.shape})
print(func_mni_img.shape == bm_func_img.shape)



#Masking
#structural image masking 
invert_mask = nimg.math_img('1-a',a= bm_t1w_img)
masked_T1_img = nimg.math_img('a * b',a = invert_mask, b= T1_mni_img)

#functional image masking 
masked_func_img = nimg.math_img('a * b',a = bm_func_img, b= func_mni_img)

#Resampling 
#resampling before adding 
resample_t1 = nimg.resample_to_img(source_img=T1_mni_img,target_img=func_mni_img,interpolation='continuous')

print(resamp_t1.shape)
print(func_mni_img.shape)

#Final Masking 
result = nimg.math_img('a + b',a = resample_t1, b= masked_func_img)

#Plotting 
nplot.plot_anat(result)
#nplot.plot_epi(result) #epi will show the clolor map

## Checking Image shape



## Slicing image 


### T1W Image Slicing

In [ ]:
#1st Method of slicing using .slice function
x_slice = T1w_mni_img.slice[10:11,:,:]

#another method of slicing using plot 
nplot.plot_anat(t1w_img,cut_coords = (50,20,30))

### fMRI image Slicing

In [ ]:

func_vol5 = func_mni_img.slicer[:,:,:,4]

#animated version of plot with slider
nplot.view_img(func_vol5)

#it is better to use epi instead of anat 
nplot.plot_epi(func_vol5)

#also we can show using the anat
nplot.plot_anat(func_vol5)

## Resampling image 



## Masking 
- As per problem 



# MRI IMAGE CLEANING
Note on - Cleaning or motion artifact 
## 1. MRI image before cleaning visualize


In [ ]:
#initial setup 
import os
from nilearn import image as nimg
from nilearn import plotting as nplot
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import bids
%matplotlib inline

import pandas as pd

In [ ]:
sub = '10788'
fmriprep_dir = '../data/ds000030/derivatives/fmriprep/'
layout = bids.BIDSLayout(fmriprep_dir,validate=False,
                        config=['bids','derivatives'])

In [ ]:
func_files = layout.get(subject=sub,
                        datatype='func', task='rest',
                        desc='preproc',
                        space='MNI152NLin2009cAsym',
                        extension='nii.gz',
                       return_type='file')

mask_files = layout.get(subject=sub,
                        datatype='func', task='rest',
                        desc='brain',
                        suffix='mask',
                        space='MNI152NLin2009cAsym',
                        extension="nii.gz",
                       return_type='file')

confound_files = layout.get(subject=sub,
                            datatype='func', task='rest',
                            desc='confounds',
                           extension="tsv",
                           return_type='file')


**Selecting Files**

In [ ]:
#selecting files
func_file = func_files[0]
mask_file = mask_files[0]
confound_file = confound_files[0]

**Display Confound file**

In [ ]:
#display counfound file 
# Using \t since it is tab-separated
confound_df = pd.read_csv(confound_file,delimiter='\t')
confound_df.head()

#confound variables 
##selecting confound variables 
confound_vars = ['trans_x','trans_y','trans_z',
                 'rot_x','rot_y','rot_z',
                 'global_signal',
                 'csf', 'white_matter']
##derivative variables 
derivative_columns = ['{}_derivative1'.format(c) for c
                     in confound_vars]

##final confound variables 
final_confounds = confound_vars + derivative_columns

#print
##confound_vars
print(f"confound list : \n {confound_vars}")

##derivative columns
print(f"derivative list : \n {derivative_columns}")

##final list 
print(f"Final Confound list : \n {final_confounds}")

#pop out columns using confound variables 
confound_df = confound_df[final_confounds]
confound_df.head()

### Dropping 3TRs 

In [ ]:
#Drop confound dummy TRs

#dropping 4 TR in dataframe 
drop_confound_df = confound_df.loc[4:]
print(drop_confound_df.shape) #number of rows should match that of the functional image
drop_confound_df.head()

#Raw image 
raw_func_img = nimg.load_img(func_file)
raw_func_img.shape

#final image after dropping 4 TRs
func_img = raw_func_img.slicer[:,:,:,4:]
func_img.shape

### Confound Regression
**Matrix Setup**

In [ ]:
#Creating the confound matrix 
##get only the values like numpy array
##drop the header 
confounds_matrix = drop_confound_df.values


**Filter Value Setup**

In [ ]:
#Set some constants
high_pass= 0.009
low_pass = 0.08
t_r = 2

**Applying Confound Regression**

In [ ]:
clean_img = nimg.clean_img(func_img,confounds=confounds_matrix,detrend=True,standardize=True,
                         low_pass=low_pass,high_pass=high_pass,t_r=t_r, mask_img=mask_file)

**Plotting**

In [ ]:
#epi plot 
nplot.plot_epi(clean_img.slicer[:,:,:,50])

#anat plot 
nplot.plot_anat(clean_img.slicer[:,:,:,50])

#Anmated plot 
##signle time point image
clean_img_slider = clean_img.slicer[:,:,:,50]
##plot using slider 
nplot.view_img(clean_img_slider, bg_img = False,cmap = 'Grey_s', symmetry_cmap = False, threshold = "auto")

## 2. MRI image after cleaning visualization
 



# MRI IMAGE PARCELLATION
## 1. Parcellation note


**Importing Modules**

In [ ]:
from nilearn import datasets
from nilearn import image as nimg
from nilearn import plotting as nplot

#for inline visualization in jupyter notebook
%matplotlib inline

**Dataset**

In [ ]:
parcel_dir = '../resources/rois/' 
atlas_yeo_2011 = datasets.fetch_atlas_yeo_2011(parcel_dir)

**Yeo2011 et. al. Labels**

In [ ]:
atlas_yeo_2011.keys()


## 2. Parcellation program - just one (as visaulzation) 

In [ ]:
cut_coords = (8, -4 ,9)
colorbar=True
cmap = 'Paired'

nplot.plot_roi(atlas_yeo_2011['thin_7'], cut_coords=cut_coords, colorbar=colorbar, cmap=cmap, title='thin_7')
nplot.plot_roi(atlas_yeo_2011['thin_17'], cut_coords=cut_coords, colorbar=colorbar, cmap=cmap, title='thin_17')
nplot.plot_roi(atlas_yeo_2011['thick_7'], cut_coords=cut_coords, colorbar=colorbar, cmap=cmap, title='thick_7')
nplot.plot_roi(atlas_yeo_2011['thick_17'], cut_coords=cut_coords, colorbar=colorbar, cmap=cmap, title='thick_17')

**Problems in Yeo2011 Brain network**

In [ ]:
atlas_yeo = atlas_yeo_2011['thick_7']

In [ ]:
Defining new network

In [ ]:
from nilearn.regions import connected_label_regions

region_labels = connected_label_regions(atlas_yeo)
nplot.plot_roi(region_labels,
              cut_coords=(-20, -10, 0, 10, 20, 30, 40, 50, 60, 70),
              display_mode='z',
              colorbar=True,
              cmap='Paired',
              title='Relabeled Yeo Atlas')

Saving to new file

In [ ]:
region_labels.to_filename('../resources/rois/yeo_2011/Yeo_JNeurophysiol11_MNI152/relabeled_yeo_atlas.nii.gz')